# Algoritmos de Inteligência Artificial para Clustericação [24E4_2]

## PROJETO DA DISCIPLINA - ENTREGA FINAL

### Aluno: Marcio Feldmann
### Repositório Público GitHub: https://github.com/marciofeld/infnet_algoritmos_clusterizacao

## Parte 1 - Infraestrutura

Para as questões a seguir, você deverá executar códigos em um notebook Jupyter, rodando em ambiente local, certifique-se que:

1. Você está rodando em Python 3.9+
2. Você está usando um ambiente virtual: Virtualenv ou Anaconda
3. Todas as bibliotecas usadas nesse exercícios estão instaladas em um ambiente virtual específico
4. Gere um arquivo de requerimentos (requirements.txt) com os pacotes necessários. É necessário se certificar que a versão do pacote está disponibilizada.
5. Tire um printscreen do ambiente que será usado rodando em sua máquina.
6. Disponibilize os códigos gerados, assim como os artefatos acessórios (requirements.txt) e instruções em um repositório GIT público. (se isso não for feito, o diretório com esses arquivos deverá ser enviado compactado no moodle).

In [ ]:
# Parte 1 - Importação de bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import dendrogram, linkage

## Parte 2 - Escolha de base de dados

Para as questões a seguir, usaremos uma base de dados e faremos a análise exploratória dos dados, antes da clusterização.

1. Baixe os dados disponibilizados na plataforma Kaggle sobre dados sócio-econômicos e de saúde que determinam o índice de desenvolvimento de um país. Esses dados estão disponibilizados através do link: [https://www.kaggle.com/datasets/rohan0301/unsupervised-learning-on-country-data](https://www.kaggle.com/datasets/rohan0301/unsupervised-learning-on-country-data)
2. Quantos países existem no dataset?
3. Mostre através de gráficos a faixa dinâmica das variáveis que serão usadas nas tarefas de clusterização. Analise os resultados mostrados. O que deve ser feito com os dados antes da etapa de clusterização?
4. Realize o pré-processamento adequado dos dados.

In [ ]:
# Parte 2 - Importação da Base de Dados
# INPUT: Country-data.csv

dataset = pd.read_csv('Country-data.csv')
dataset.head()

In [ ]:
# Parte 2 - Exercício 2

qnt_paises = dataset['country'].nunique()
print('\033[1mParte 2 - Exercício 2 - Quantos países existem no dataset?\033[0m\n')
print(f'Resposta do Exercício 2: \033[1m{qnt_paises}\033[0m')

In [ ]:
# Definição das colunas de dados

colunas_de_dados = ['child_mort','exports','health','imports','income','inflation','life_expec','total_fer','gdpp']

In [ ]:
# Parte 2 - Exercício 3 - Mostre através de gráficos a faixa dinâmica das variáveis que serão utilizadas nas tarefas de clusterização.

n_linhas = 3
n_colunas = 3

fig, axes = plt.subplots(n_linhas, n_colunas, figsize=(15, 12), sharey=True)
axes = axes.flatten()

for i, coluna in enumerate(colunas_de_dados):
    axes[i].boxplot(dataset[coluna])
    axes[i].set_title(coluna)
    axes[i].set_xlabel(coluna)

fig.supylabel('Valores Originais')
fig.suptitle('Parte 2 - Exercício 3 - Faixa Dinâmica das Variáveis', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Parte 2 - Exercício 3 - Analise os resultados mostrados. O que deve ser feito com os dados antes da etapa de clusterização?

print('\033[1mParte 2 - Exercício 3 - Resposta\033[0m\n')
print('Após a análise dos resultados da faixa dinâmica das variáveis, verifico que devemos executar uma normalização de todas as colunas de dados na etapa de pré-processamento.')

In [ ]:
# Parte 2 - Exercício 4 - Realize o pré-processamento adequado dos dados (1)

# Mapeando nomes de países para números
pais_para_numero = {country: i for i, country in enumerate(dataset['country'])}
# Mapeando números para nomes de países
numero_para_pais = {i: country for country, i in pais_para_numero.items()}

print('\033[1mDicionário: pais_para_numero\033[0m')
print(pais_para_numero)
print('\033[1mDicionário: numero_para_pais\033[0m')
print(numero_para_pais)

In [ ]:
# Parte 2 - Exercício 4 - Realize o pré-processamento adequado dos dados (2)

# Realizando normalização de dados
scaler = MinMaxScaler()
dataset_normalizado = scaler.fit_transform(dataset[colunas_de_dados])
dataset_normalizado = pd.DataFrame(dataset_normalizado, columns=colunas_de_dados)

# Inserindo na primeira coluna do `dataset_normalizado` os códigos numéricos de cada país
dataset_normalizado.insert(0, 'country_code', dataset['country'].map(pais_para_numero))

print('\033[1mDataset Normalizado com códigos numéricos para os países:\033[0m')
print(dataset_normalizado)

In [ ]:
# Parte 2 - Exercício 4 - Realize o pré-processamento adequado dos dados (3)

n_linhas = 3
n_colunas = 3

fig, axes = plt.subplots(n_linhas, n_colunas, figsize=(15, 12), sharey=True)
axes = axes.flatten()

for i, coluna in enumerate(colunas_de_dados):
    axes[i].boxplot(dataset_normalizado[coluna])
    axes[i].set_title(coluna)
    axes[i].set_xlabel(coluna)

fig.supylabel('Valores Normalizados')
fig.suptitle('Parte 2 - Exercício 4 - Dados Normalizados', fontsize=16)
plt.tight_layout()
plt.show()

## Parte 3 - Clusterização

Para os dados pré-processados da etapa anterior você irá:

1. Realizar o agrupamento dos países em 3 grupos distintos. Para tal, use:
    - a. K-Médias
    - b. Clusterização Hierárquica
2. Para os resultados, do K-Médias:
    - a. Interprete cada um dos clusters obtidos citando:
        - i. Qual a distribuição das dimensões em cada grupo
        - ii. O país, de acordo com o algoritmo, melhor representa o seu agrupamento. Justifique
3. Para os resultados da Clusterização Hierárquica, apresente o dendograma e interprete os resultados
4. Compare os dois resultados, aponte as semelhanças e diferenças e interprete.

In [ ]:
# Definição de número de clusters

n_clusters=3

In [ ]:
# Parte 3 - Exercício 1.a - Agrupamento dos países em 3 grupos distintos usando K-Médias

kmeans = KMeans(n_clusters=n_clusters,random_state=20)
dataset_normalizado['Cluster_Kmeans'] = kmeans.fit_predict(dataset_normalizado[colunas_de_dados])

print('\033[1mParte 3 - Exercício 1.a - Agrupamento dos países em 3 grupos distintos usando K-Médias\033[0m')
print(
    pd.DataFrame({
        'Country': dataset_normalizado['country_code'].map(numero_para_pais),
        'Cluster_Kmeans': dataset_normalizado['Cluster_Kmeans']
    })
)

In [ ]:
# Parte 3 - Exercício 1.b - Agrupamento dos países em 3 grupos distintos usando Clusterização Hierárquica

data = dataset_normalizado[colunas_de_dados].values

clusterizacao_hierarquica = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
dataset_normalizado['Cluster_Hierarquico'] = clusterizacao_hierarquica.fit_predict(data)

print('\033[1mParte 3 - Exercício 1.b - Agrupamento dos países em 3 grupos distintos usando Clusterização Hierárquica\033[0m')
print(
    pd.DataFrame({
        'Country': dataset_normalizado['country_code'].map(numero_para_pais),
        'Cluster_Hierarquico': dataset_normalizado['Cluster_Hierarquico']
    })
)

In [ ]:
# Parte 3 - Exercício 2.a.i - Para os resultados do K-Médias, interprete cada um dos clusters obtidos citando: Qual a distribuição das dimensões em cada grupo (1)

distribuicao_clusters = dataset_normalizado.groupby('Cluster_Kmeans').mean()
print('\033[1mParte 3 - Exercício 2.a.i(1) - Distribuição das dimensões em cada grupo utilizando o K-Médias\033[0m')
print(distribuicao_clusters.iloc[:, 1:10])

In [ ]:
# Parte 3 - Exercício 2.a.i - Para os resultados do K-Médias, interprete cada um dos clusters obtidos citando: Qual a distribuição das dimensões em cada grupo (2)

print('\033[1mParte 3 - Exercício 2.a.i - Para os resultados do K-Médias, interprete cada um dos clusters obtidos citando: Qual a distribuição das dimensões em cada grupo\033[0m\n')

def classificar_cluster(valores):
    # Classificar os clusters automaticamente com base em dimensões-chave (income, life_expec e child_mort)
    ## Países desenvolvidos possuem alta renda (income), alta expectativa de vida (life_expec) e baixa mortalidade infantil (child_mort)
    if valores['income'] > 0.3 and valores['life_expec'] > 0.9 and valores['child_mort'] < 0.05:
        return 'Países desenvolvidos'
    ## Países subdesenvolvidos possuem baixa renda (income), baixa expectativa de vida (life_expec) e alta mortalidade infantil (child_mort)
    elif valores['income'] < 0.1 and valores['life_expec'] < 0.6 and valores['child_mort'] > 0.4:
        return 'Países subdesenvolvidos'
    ## Países em desenvolvimento possuem valores intermediários
    else:
        return 'Países em desenvolvimento'

def interpretar_clusters(distribuicao):
    interpretacoes = []
    for cluster, valores in distribuicao.iterrows():
        tipo_cluster = classificar_cluster(valores)
        interpretacao = (
            f'\033[1mCluster {cluster}: {tipo_cluster}\033[0m\n'
            f'- Mortalidade infantil: {valores['child_mort']:.4f}\n'
            f'- Expectativa de vida: {valores['life_expec']:.4f}\n'
            f'- Renda per capita: {valores['income']:.4f}\n'
            f'- Inflação: {valores['inflation']:.4f}\n'
            f'- PIB: {valores['gdpp']:.4f}\n'
        )
        interpretacoes.append(interpretacao)
    return interpretacoes

interpretacoes = interpretar_clusters(distribuicao_clusters)

for interpretacao in interpretacoes:
    print(interpretacao)


In [ ]:
# Parte 3 - Exercício 2.a.ii - Para os resultados do K-Médias, interprete cada um dos clusters obtidos citando: O país, de acordo com o algoritmo, melhor representa o seu agrupamento. Justifique (1)

centroides = kmeans.cluster_centers_
distancias = cdist(dataset_normalizado[colunas_de_dados], centroides, metric='euclidean')

representantes = {}
for cluster in range(n_clusters):
    indices_cluster = dataset_normalizado['Cluster_Kmeans'] == cluster
    distancias_cluster = distancias[indices_cluster, cluster]
    indice_mais_proximo = np.argmin(distancias_cluster)
    pais_mais_proximo = dataset_normalizado.loc[indices_cluster].iloc[indice_mais_proximo]['country_code']
    representantes[cluster] = numero_para_pais[pais_mais_proximo]

print('\033[1mParte 3 - Exercício 2.a.ii(1) - Distribuição das dimensões em cada grupo utilizando o K-Médias\033[0m')
print('\033[1mPaíses que melhor representam cada agrupamento:\033[0m\n')
for cluster, pais in representantes.items():
    print(f'\033[1mCluster {cluster}:\033[0m {pais}')

In [ ]:
# Parte 3 - Exercício 2.a.ii - Para os resultados do K-Médias, interprete cada um dos clusters obtidos citando: O país, de acordo com o algoritmo, melhor representa o seu agrupamento. Justifique (2)

print('\033[1mParte 3 - Exercício 2.a.ii(2) - Distribuição das dimensões em cada grupo utilizando o K-Médias - Justificativas\033[0m\n')

def justificar_representantes(representantes, centroides, dataset, colunas, numero_para_pais):
    justificativas = []
    
    for cluster, pais in representantes.items():
        country_code = list(numero_para_pais.keys())[list(numero_para_pais.values()).index(pais)]
        valores_pais = dataset[dataset['country_code'] == country_code][colunas].iloc[0]
        valores_centroide = centroides[cluster]
        diferencas = abs(valores_pais - valores_centroide)
        
        menores_indices = diferencas.argsort()[:3]
        menores_colunas = [colunas[i] for i in menores_indices]
        menores_diferencas = [diferencas.iloc[i] for i in menores_indices]
        
        justificativa = (
            f'\033[1mCluster {cluster} - Representante: {pais}\033[0m\n'
            f'Este país foi escolhido porque possui valores próximos ao centróide do cluster.\n'
            f'As 3 dimensões com menor diferença absoluta entre o país \033[1m{pais}\033[0m e o centróide do cluster são:\n'
        )
        
        for coluna, diferenca in zip(menores_colunas, menores_diferencas):
            justificativa += f'- {coluna}: {diferenca:.4f}\n'
        
        justificativas.append(justificativa)
    
    return justificativas

justificativas = justificar_representantes(representantes, centroides, dataset_normalizado, colunas_de_dados, numero_para_pais)

for justificativa in justificativas:
    print(justificativa)


In [ ]:
# Parte 3 - Exercício 3 - Para os resultados da Clusterização Hierárquica, apresente o dendograma e interprete os resultados (1)

# Criar a matriz de linkagem para o dendrograma usando o método 'ward'
linkage_matrix = linkage(data, method='ward')

# Função para plotar o dendrograma
plt.figure(figsize=(15, 8))
plt.title('Parte 3 - Exercício 3 - Dendograma da Clusterização Hierárquica', fontsize=16)
plt.xlabel('Países', fontsize=12)
plt.ylabel('Distância Euclidiana', fontsize=12)

dendrogram(
    linkage_matrix,
    labels=dataset_normalizado['country_code'].map(numero_para_pais).values, leaf_rotation=90, leaf_font_size=6,
)

plt.tight_layout()
plt.show()

In [ ]:
# Parte 3 - Exercício 3 - Para os resultados da Clusterização Hierárquica, apresente o dendograma e interprete os resultados (2)

# Listar os países em cada cluster hierárquico
for cluster in sorted(dataset_normalizado['Cluster_Hierarquico'].unique()):
    print(f'Cluster {cluster}:')
    print(dataset_normalizado[dataset_normalizado['Cluster_Hierarquico'] == cluster]['country_code']
          .map(numero_para_pais)
          .tolist())
    print()


In [ ]:
# Parte 3 - Exercício 3 - Para os resultados da Clusterização Hierárquica, apresente o dendograma e interprete os resultados.

print('## Justificativa dos Clusters - Dendograma da Clusterização Hierárquica')

## Cluster 1
print('\n\033[1m### Cluster 1 (Laranja - Subdesenvolvidos)\033[0m')
print('- \033[1mPaíses incluídos\033[0m: Camarões, Angola, Serra Leoa, Congo, entre outros.')
print('- \033[1mJustificação\033[0m:')
print('  - Esses países compartilham baixos indicadores socioeconômicos.')
print('  - \033[1mCaracterísticas principais\033[0m:')
print('    - \033[1mRenda per capita\033[0m: Baixa.')
print('    - \033[1mMortalidade infantil\033[0m: Alta.')
print('    - \033[1mExpectativa de vida\033[0m: Baixa.')

## Cluster 2
print('\n\033[1m### Cluster 2 (Verde à esquerda - Em Desenvolvimento)\033[0m')
print('- \033[1mPaíses incluídos\033[0m: Micronésia, Brasil, Paraguai, entre outros.')
print('- \033[1mJustificação\033[0m:')
print('  - Estes países possuem características intermediárias, tanto sociais quanto econômicas.')
print('  - \033[1mCaracterísticas principais\033[0m:')
print('    - \033[1mRenda per capita\033[0m: Moderada.')
print('    - \033[1mMortalidade infantil\033[0m: Moderada.')
print('    - \033[1mExpectativa de vida\033[0m: Moderada.')

## Cluster 3
print('\n\033[1m### Cluster 3 (Verde à direita - Desenvolvidos)\033[0m')
print('- \033[1mPaíses incluídos\033[0m: Japão, Estados Unidos, Alemanha, entre outros.')
print('- \033[1mJustificação\033[0m:')
print('  - Esses países têm indicadores socioeconômicos elevados, refletindo alto desenvolvimento.')
print('  - \033[1mCaracterísticas principais\033[0m:')
print('    - \033[1mRenda per capita\033[0m: Alta.')
print('    - \033[1mMortalidade infantil\033[0m: Baixa.')
print('    - \033[1mExpectativa de vida\033[0m: Alta.')

## Observações gerais
print('\n\033[1m### Observações gerais\033[0m')
print('- A \033[1maltura das uniões\033[0m no dendograma reflete as diferenças entre os grupos:')
print('  - Clusters unidos em alturas maiores são mais heterogêneos.')
print('  - Clusters unidos em alturas menores são mais homogêneos.')
print('- A separação em 3 grandes clusters foi realizada com base em uma altura de corte de aproximadamente \033[1m2.5\033[0m no eixo Y.')


In [ ]:
# Parte 3 - Exercício 4 - Compare os dois resultados, aponte as semelhanças e diferenças e interprete. (1)
# OUTPUT: parte3_exercicio4_comparacao_clusters.csv

comparacao_clusters = dataset_normalizado[['country_code', 'Cluster_Kmeans', 'Cluster_Hierarquico']].copy()
comparacao_clusters.loc[:, 'Country'] = comparacao_clusters['country_code'].map(numero_para_pais)
comparacao_clusters = comparacao_clusters[['Country', 'Cluster_Kmeans', 'Cluster_Hierarquico']]

# Adicionar uma coluna indicando se os clusters coincidem
comparacao_clusters['Mesmo_Cluster'] = comparacao_clusters['Cluster_Kmeans'] == comparacao_clusters['Cluster_Hierarquico']

# Exibir os primeiros 20 resultados
print('\033[1mParte 3 - Exercício 4 - Tabela comparativa\033[0m\n')
print(comparacao_clusters.head(20))

# Opcional: salvar a tabela de comparação em um arquivo CSV
comparacao_clusters.to_csv('parte3_exercicio4_comparacao_clusters.csv', index=False)
print('Tabela de comparação salva como "parte3_exercicio4_comparacao_clusters.csv"')

In [ ]:
# Parte 3 - Exercício 4 - Compare os dois resultados, aponte as semelhanças e diferenças e interprete. (2)

# Calcular estatísticas para a interpretação
total_paises = len(comparacao_clusters)
contagem_mesmo_cluster = comparacao_clusters['Mesmo_Cluster'].sum()
proporcao_mesmo_cluster = contagem_mesmo_cluster / total_paises

# Países com clusters diferentes
clusters_diferentes = comparacao_clusters[comparacao_clusters['Mesmo_Cluster'] == False]

# Exibir a interpretação
print('\033[1mParte 3 - Exercício 4 - Diferenças\033[0m\n')
print(f'Total de países analisados: {total_paises}')
print(f'Número de países no mesmo cluster nos dois métodos: {contagem_mesmo_cluster}')
print(f'Proporção de países no mesmo cluster: {proporcao_mesmo_cluster:.2%}\n')

if len(clusters_diferentes) > 0:
    print('Países classificados em clusters diferentes:')
    print(clusters_diferentes[['Country', 'Cluster_Kmeans', 'Cluster_Hierarquico']].to_string(index=False))
else:
    print('Todos os países foram classificados no mesmo cluster nos dois métodos.')

In [ ]:
# Parte 3 - Exercício 4 - Compare os dois resultados, aponte as semelhanças e diferenças e interprete. (3)
# OUTPUT: parte3_exercicio4justificacao_diferencas_clusters.csv

# Selecionar os países classificados de forma diferente
diferencas_clusters = comparacao_clusters[comparacao_clusters['Mesmo_Cluster'] == False]

# Obter os dados completos desses países usando a coluna 'Country'
diferencas_dados = dataset_normalizado[dataset_normalizado['country_code'].map(numero_para_pais).isin(diferencas_clusters['Country'])]

# Adicionar os clusters K-Means e Hierárquico para comparação
diferencas_dados = diferencas_dados[['country_code'] + colunas_de_dados + ['Cluster_Kmeans', 'Cluster_Hierarquico']]
diferencas_dados['Country'] = diferencas_dados['country_code'].map(numero_para_pais)

# Reorganizar as colunas para melhor visualização
diferencas_dados = diferencas_dados[['Country', 'Cluster_Kmeans', 'Cluster_Hierarquico'] + colunas_de_dados]

# Analisar as diferenças entre os clusters
print('\033[1mParte 3 - Exercício 4 - Interpretação\033[0m')
for _, row in diferencas_dados.iterrows():
    print(f'\033[1m\nPaís: {row["Country"]}\033[0m')
    print(f'Cluster K-Means: {row["Cluster_Kmeans"]} | Cluster Hierárquico: {row["Cluster_Hierarquico"]}')
    print('Características principais:')
    for coluna in colunas_de_dados:
        print(f'- {coluna}: {row[coluna]:.4f}')
    print('Justificação:')
    if row['Cluster_Kmeans'] != row['Cluster_Hierarquico']:
        print('-> Diferença de classificação: Este país apresenta características que estão \'na fronteira\' entre os clusters,'
              ' o que leva os dois métodos a classificá-lo de maneira distinta com base nas distâncias e abordagens.')
    print('-' * 50)

# Salvar as diferenças em um arquivo CSV para análise adicional
diferencas_dados.to_csv('parte3_exercicio4_justificacao_diferencas_clusters.csv', index=False)
print('\nArquivo \'parte3_exercicio4_justificacao_diferencas_clusters.csv\' criado com os dados detalhados.')


## Parte 4 - Escolha de Algoritmos

1. Escreva em tópicos as etapas do algoritmo de K-médias até sua convergência.
2. O algoritmo de K-médias converge até encontrar os centróides que melhor descrevem os clusters encontrados (até o deslocamento entre as interações dos centróides ser mínimo). Lembrando que o centróide é o baricentro do cluster em questão e não representa, em via de regra, um dado existente na base. Refaça o algoritmo apresentado na questão 1 a fim de garantir que o cluster seja representado pelo dado mais próximo ao seu baricentro em todas as iterações do algoritmo.
    - Obs: nesse novo algoritmo, o dado escolhido será chamado medóide.
3. O algoritmo de K-médias é sensível a outliers nos dados. Explique.
4. Por que o algoritmo de DBScan é mais robusto à presença de outliers?

In [ ]:
# Parte 4 - Exercício 1 - Escreva em tópicos as etapas do algoritmo de K-médias até sua convergência.

print('\033[1mParte 4 - Exercício 1 - Escreva em tópicos as etapas do algoritmo de K-médias até sua convergência.o\033[0m')

print('\n\033[1mEtapa 1: Escolha do número de grupos (clusters).\033[0m')
print('* Decisão do número de grupos para dividir os dados. Esse número é chamado de k.')

print('\n\033[1mEtapa 2: Posicionamento inicial dos centros (centróides).\033[0m')
print('* Os centróides iniciais são escolhidos automaticamente pelo algoritmo de K-Means.')
print('* No código, usei "KMeans(n_clusters=n_clusters, random_state=20)" para definir o número de clusters e garantir reprodutibilidade.')

print('\n\033[1mEtapa 3: Agrupamento inicial dos pontos.\033[0m')
print('* Para cada ponto nos dados:')
print('   - O algoritmo calcula qual centróide está mais próximo.')
print('   - O ponto é então atribuído ao grupo (cluster) correspondente.')

print('\n\033[1mEtapa 4: Ajuste dos centros dos grupos.\033[0m')
print('* Após a atribuição inicial:')
print('   - O algoritmo calcula o "novo centro" de cada grupo.')
print('   - Este novo centro é a posição média de todos os pontos atribuídos ao grupo.')
print('   - Os centróides são ajustados para essas novas posições.')

print('\n\033[1mEtapa 5: Reagrupamento dos pontos.\033[0m')
print('* Com os centróides ajustados:')
print('   - Cada ponto é reavaliado para verificar qual centróide está mais próximo.')
print('   - Se algum ponto mudar de grupo, o agrupamento é atualizado.')

print('\n\033[1mEtapa 6: Repetição até estabilizar.\033[0m')
print('* O processo de ajuste e reagrupamento continua até que:')
print('   - Nenhum ponto mude de grupo.')
print('   - Ou o algoritmo atinja um número máximo de iterações.')

print('\n\033[1mEtapa 7: Resultados finais.\033[0m')
print('* No final, o algoritmo retorna:')
print('   - Todos os dados divididos em k grupos.')
print('   - Os centróides finais, que representam o "coração" de cada grupo.')


In [ ]:
# Parte 4 - Exercício 2 - O algoritmo de K-médias converge até encontrar os centróides que melhor descrevem os clusters encontrados (até o deslocamento entre as interações dos centróides ser mínimo).
# Lembrando que o centróide é o baricentro do cluster em questão e não representa, em via de regra, um dado existente na base.
# Refaça o algoritmo apresentado na questão 1 a fim de garantir que o cluster seja representado pelo dado mais próximo ao seu baricentro em todas as iterações do algoritmo.
# Obs: nesse novo algoritmo, o dado escolhido será chamado medóide.

# Inicializar os medóides com índices aleatórios
np.random.seed(20)
indices_iniciais = np.random.choice(len(dataset_normalizado), size=n_clusters, replace=False)
medoids = dataset_normalizado[colunas_de_dados].iloc[indices_iniciais].values

# Função para calcular clusters e custo
def atribuir_clusters_e_calcular_custo(data, medoids):
    distancias = cdist(data, medoids, metric='euclidean')  # Distâncias Euclidianas
    clusters = np.argmin(distancias, axis=1)  # Atribuir ao medóide mais próximo
    custo = np.sum(np.min(distancias, axis=1))  # Soma das distâncias mínimas
    return clusters, custo

# Iterar até convergência
medoide_anterior = None
iteracao_kmedoid = 0
while not np.array_equal(medoide_anterior, medoids):
    iteracao_kmedoid += 1
    medoide_anterior = medoids.copy()
    # Atribuir clusters e calcular o custo
    dataset_normalizado['Cluster_Kmedoids'], _ = atribuir_clusters_e_calcular_custo(
        dataset_normalizado[colunas_de_dados].values, medoids
    )
    # Atualizar medóides para o ponto mais central do cluster
    for cluster in range(n_clusters):
        cluster_points = dataset_normalizado[dataset_normalizado['Cluster_Kmedoids'] == cluster][colunas_de_dados].values
        medoid_index = np.argmin(np.sum(cdist(cluster_points, cluster_points, metric='euclidean'), axis=1))
        medoids[cluster] = cluster_points[medoid_index]

# Exibir os resultados
print('\033[1mParte 4 - Exercício 2 - Utilização de K-medoids\033[0m\n')

print(f'Convergência alcançada após \033[1m{iteracao_kmedoid}\033[0m iterações\n')

print(
    pd.DataFrame({
        'Country': dataset_normalizado['country_code'].map(numero_para_pais),
        'Cluster_Kmedoids': dataset_normalizado['Cluster_Kmedoids']
    })
)



In [ ]:
# Parte 4 - Exercício 3 - O algoritmo de K-médias é sensível a outliers nos dados. Explique.

print('\033[1mParte 4 - Exercício 3 - O algoritmo de K-médias é sensível a outliers nos dados. Explique.\033[0m\n')

print('\033[1mPor que o K-Means é sensível a outliers?\033[0m')
print('* O algoritmo calcula os centróides como a média dos pontos no cluster.')
print('* Outliers, que são valores muito distantes, podem deslocar o centróide, tornando-o menos representativo para a maioria dos pontos.')

print('\n\033[1mImpacto nos agrupamentos:\033[0m')
print('* Os clusters podem ficar distorcidos, não refletindo bem os dados.')
print('* Em alguns casos, os outliers podem até formar clusters próprios, atrapalhando a análise.')

print('\n\033[1mComo evitar o problema?\033[0m')
print('\033[1m1.\033[0m Remover ou tratar os outliers antes de rodar o algoritmo.')
print('\033[1m2.\033[0m Normalizar os dados para reduzir o impacto de valores extremos.')
print('\033[1m3.\033[0m Considerar algoritmos mais robustos, como o K-Medoids, que usa pontos reais como representantes dos clusters.')

print('\n\033[1mFontes:\033[0m')
print('* Sensibilidade do K-Médias: https://scikit-learn.org/stable/modules/clustering.html#k-means')
print('* Comparação K-Médias e K-Medoids: https://www.geeksforgeeks.org/k-means-vs-k-medoids-clustering/')
print('* Lidando com Outliers: https://www.datageeks.com.br/outliers/')


In [ ]:
# Parte 4 - Exercício 4 - Por que o algoritmo de DBScan é mais robusto à presença de outliers?

print('\033[1mParte 4 - Exercício 4 - Por que o algoritmo de DBScan é mais robusto à presença de outliers?\033[0m\n')

print('\033[1mPor que o DBSCAN é mais robusto a outliers?\033[0m')
print('* O algoritmo identifica automaticamente os outliers como pontos que não pertencem a nenhum cluster.')
print('* Ele forma clusters com base na densidade de pontos, ignorando os que estão isolados.')
print('* Diferente do K-Means, o DBSCAN não usa centróides, então outliers não deslocam os clusters.')
print('* Pontos isolados são classificados como ruído e não afetam o agrupamento.')

print('\n\033[1mFontes:\033[0m')
print('* Robustez do DBSCAN: https://scikit-learn.org/stable/modules/clustering.html#dbscan')
print('* Comparação entre DBSCAN e K-Means: https://towardsdatascience.com/dbscan-clustering-explained-97556a2ad556')
print('* Lidando com Outliers: https://www.datageeks.com.br/outliers/')
